# Hybrid bias emulator for Brans-Dicke

- [ ] Emulate BD/GR ratio and multiply up with Bacco output?
- [ ] Which subset of parameters? Idea: plot $P_l(k,z)$ for all LHS samples, then color according to each parameter to see variation in all of parameter space
- [ ] Check convergence of COLA
- [ ] Use same parameters in CLASS and COLA
- [ ] Set COLA hyperparameters
- [ ] Calculate power spectrum multipoles on the fly (Gui?)
- [ ] Decide optimal parameterization of parameter space (e.g. $\log \omega$ instead of $\omega$, $\sigma_8$ instead of $A_s$, make sound horizon equal?)
- [ ] https://nbodykit.readthedocs.io/en/latest/cookbook/fftpower.html?highlight=multipoles
- [ ] Massive neutrinos?
- [ ] $w_0 w_a$? Implemented in COLA BD?


In [ ]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import pandas
import seaborn
import json
import subprocess
import re
from pathlib import Path
from hashlib import md5

## Simulation setup

1. Run CLASS
2. Run COLA

In [ ]:
DATADIR = "data" # relative path
CLASSEXEC = "/mn/stornext/u3/hermasl/hybrid-bias-cola-emulator/hi_class_public/class" # absolute path
COLAEXEC = "/mn/stornext/u3/hermasl/hybrid-bias-cola-emulator/FML/FML/COLASolver/nbody" # absolute path

# Deterministic hashing based on https://stackoverflow.com/a/10288255
def hash_data(p):
    return md5(json.dumps(p, sort_keys=True).encode("utf-8")).hexdigest()

def read_number(filename, prefix):
    regex = prefix + r"([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"
    with open(filename, "r") as f:
        for line in f:
            match = re.search(regex, line)
            if match:
                return float(match.group(1))
    raise f"did not find 1 match for regex {regex} in {filename}"

def get_model(p):
    return "BD" if "ω" in p else "GR"

def input_class(p):
    # See https://github.com/emiliobellini/hi_class_public/blob/hi_class/explanatory.ini
    # See https://github.com/emiliobellini/hi_class_public/blob/hi_class/gravity_models/brans_dicke.ini
    lines = [
        # varying cosmological parameters
        f"Omega_m = {p['Ωm']}",
        f"sigma8 = {p['σ8']}", # not sigma_8
        f"Omega_b = {p['Ωb']}",
        f"n_s = {p['ns']}",
        f"h = {p['h']}",
        f"m_ncdm = 0.0", # TODO: {p['Mν']}",
        f"w0_fld = 0.0", # TODO: {p['w0']}",
        f"wa_fld = 0.0", # TODO: {p['wa']}",

        # fixed cosmological parameters
        f"N_eff = 3.046",
        f"T_cmb = 2.7255",
        f"Omega_k = 0.0",

        # output control
        f"root = ./", # use one folder only
        f"overwrite_root = yes", # don't create numbered files
        f"output = mPk", # output matter power spectrum P(k,z)
        f"write_background = yes", # output table of background functions
        f"output_background_smg = 2", # include phi in background table
        f"input_verbose = 1", # write As(σ8)
        f"background_verbose = 3", # write modified gravity information (e.g. Lambda)
    ]
    model = get_model(p)
    if model == "GR":
        pass
    elif model == "BD":
        lines.extend([
            f"gravity_model = brans_dicke",
            f"Omega_Lambda = 0",
            f"Omega_fld = 0",
            f"Omega_smg = -1",
            f"M2_tuning_smg = yes",
            f"M2_today_smg = {(4+2*p['ω'])/(3+2*p['ω'])}", # TODO: G0 ≠ 1?
            f"parameters_smg = NaN, {p['ω']}, 1.0, 1.0", # V₀ (shooting from CLASS' internal guess), ω, ϕᵢ (shooting from this guess), ϕᵢ' (must be nonzero)
            f"pert_initial_conditions_smg = zero",
            f"a_min_stability_test_smg = 1e-6",
        ])
    else:
        raise f"unknown model {model}"
        
    return '\n'.join(lines) + '\n' # final newline

def input_cola(p):
    # See https://github.com/HAWinther/FML/blob/master/FML/COLASolver/parameterfile.lua
    lines = [
        f"all_parameters_must_be_in_file = false",

        # varying cosmological parameters
        f"cosmology_OmegaCDM = {p['Ωm']-p['Ωb']}",
        f"cosmology_As = {p['As']}", # found from σ8 in CLASS
        f"cosmology_Omegab = {p['Ωb']}",
        f"cosmology_ns = {p['ns']}",
        f"cosmology_h = {p['h']}",

        # fixed cosmological parameters
        f"cosmology_Neffective = 3.046",
        f"cosmology_TCMB_kelvin = 2.7255",
        f"cosmology_OmegaK = 0.0",
        f"cosmology_OmegaMNu = 0.0", # TODO
        # TODO: kpivot

        # initial conditions
        f"ic_type_of_input = \"powerspectrum\"",
        f"ic_input_filename = \"../class/_pk.dat\"",
        f"ic_input_redshift = 0.0",
        f"ic_random_field_type = \"gaussian\"",
        f"ic_random_seed = 123",
        f"ic_fix_amplitude = true",
        f"ic_initial_redshift = 9.0",
        f"ic_nmesh = 128",

        # computational hyperparameters # TODO
        f"simulation_name = \"cola\"",
        f"simulation_boxsize = 128.0",
        f"particle_Npart_1D = 128",
        f"timestep_nsteps = {{{10}}}",
        f"force_nmesh = 128",

        # output
        f"output_folder = \"output\"",
        f"output_redshifts = {{{0.0}}}",
        f"output_particles = true",
        f"output_fileformat = \"GADGET\"",
    ]
    model = get_model(p)
    if model == "GR":
        lines.extend([
            f"gravity_model = \"GR\"",
            f"cosmology_model = \"LCDM\"",
        ])
    elif model == "BD":
        lines.extend([
            f"gravity_model = \"JBD\"",
            f"cosmology_model = \"JBD\"",
            f"cosmology_JBD_wBD = {p['ω']}",
            f"cosmology_JBD_GeffG_today = 1.0", # TODO: G0 ≠ 1?
            f"cosmology_JBD_density_parameter_definition = \"hi-class\"",
        ])
    else:
        raise f"unknown model {model}"
    return '\n'.join(lines) + '\n' # final newline

def run_command(cmd, np=None, dir=None, capture=True, dry=False, verbose=False, check=True):
    if np is not None:
        cmd = f"mpirun -np {np} {cmd}"
    if verbose:
        print(f"Running {cmd} in {dir}")
    if not dry:
        return subprocess.run(cmd, shell=True, cwd=dir, check=check, capture_output=capture, stdin=subprocess.DEVNULL) # https://stackoverflow.com/a/45988305

def get_derived_parameters(dir):
    classlog = f"{dir}/class/log.txt"
    return {
        "As": read_number(classlog, "A_s = "),
    }

def parameter_summary(p):
    return ", ".join(f"{k}={p[k]:+.2e}" for k in p.keys())

# TODO: pair BD vs. GR
def simulate(p0):
    p = dict(p0) # p0 is only independent parameters, p can also contain dependent parameters
    hash = hash_data(p)

    complete = Path(f"{DATADIR}/{hash}/parameters.json").exists()
    print(f"hash={hash[0:5]}…,", parameter_summary(p), "(already run)" if complete else "")

    if not complete:
        # 1) Create directory for this independent parameter sample
        Path(f"{DATADIR}/{hash}/").mkdir(exist_ok=True, parents=True)

        # 2) Set up and run CLASS
        Path(f"{DATADIR}/{hash}/class").mkdir(exist_ok=True)
        Path(f"{DATADIR}/{hash}/class/input.ini").write_text(input_class(p))
        run = run_command(f"{CLASSEXEC} input.ini", dir=f"{DATADIR}/{hash}/class/", check=True, capture=True, verbose=True)
        Path(f"{DATADIR}/{hash}/class/log.txt").write_bytes(run.stdout)

        # 3) Derive dependent parameters from CLASS
        pext = get_derived_parameters(f"{DATADIR}/{hash}")
        p |= pext # merge into p

        # 4) Set up and run COLA
        Path(f"{DATADIR}/{hash}/cola").mkdir(exist_ok=True)
        Path(f"{DATADIR}/{hash}/cola/input.lua").write_text(input_cola(p))
        run = run_command(f"{COLAEXEC} input.lua", dir=f"{DATADIR}/{hash}/cola/", check=True, capture=True, verbose=True)
        Path(f"{DATADIR}/{hash}/cola/log.txt").write_bytes(run.stdout)

        # 5) Write all parameters to file to indicate successful run
        Path(f"{DATADIR}/{hash}/parameters.json").write_text(json.dumps(p, ensure_ascii=False, indent=4)) # create if successful

    return hash

## Fiducial cosmology

In [ ]:
pfid = {"ω": 20.0, "Ωm": 0.40, "σ8": 0.90, "Ωb": 0.06, "ns": 1.0, "h": 0.7, "Mν": 0.0, "w0": -1.0, "wa": 0.0}
hashfid = simulate(pfid)
hashfid

In [ ]:
def read_table(filename, header=1):
    with open(filename, "r") as f:
        for row in range(header):
            line = next(f)
        columns = re.split("\s\s+", line)[1:] # skip first "#"
        columns = [col.strip() for col in columns]
        if not columns[-1]:
            columns = columns[:-1] # remove last column if empty string (due to CLASS' messed up header format)
        print(len(columns), "columns:", ", ".join(columns))
        return pandas.read_table(f, sep="\s+", names=columns, header=None) # reads remaining lines only

dir = f"data/{hashfid}"
data_class = read_table(f"{dir}/class/_background.dat", header=4)
data_class = data_class.loc[1 / (1 + data_class["1:z"]) > 1e-9] # remove scale factors before COLA
data_class

In [ ]:
data_cola = read_table(f"{dir}/cola/output/cosmology_cola.txt")
data_cola = data_cola.loc[data_cola["a"] <= 1] # remove scale factors after CLASS
data_cola

In [ ]:
alpha = 0.5
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (20, 4), sharex = True)

a_cola = data_cola["a"]
a_class = 1 / (1 + data_class["1:z"])
E_cola = data_cola["H/H0"]
E_class = data_class["4:H [1/Mpc]"]/data_class["4:H [1/Mpc]"].iloc[-1]
ax1.plot(np.log10(a_cola), np.log10(E_cola * a_cola**1), linestyle = "solid", color = "black", alpha = alpha)
ax1.plot(np.log10(a_class), np.log10(E_class * a_class**1), linestyle = "dashed", color = "black", alpha = alpha)
ax1.hlines([np.nan], xmin=-9, xmax=0, color = "black", linestyle = "solid", alpha = alpha, label = "COLA")
ax1.hlines([np.nan], xmin=-9, xmax=0, color = "black", linestyle = "dashed", alpha = alpha, label = "CLASS")
ax1.set_xlabel("log10(a)")
ax1.set_ylabel("aH / H0")
ax1.set_xlim(-9, 0)
ax1.legend()

φ_cola = data_cola["phi"]
φ_class = data_class["34:phi_smg"]
ax2.plot(np.log10(a_cola), φ_cola, linestyle = "solid", color = "black", alpha = alpha)
ax2.plot(np.log10(a_class), φ_class, linestyle = "dashed", color = "black", alpha = alpha)
ax2.set_xlabel("log10(a)")
ax2.set_ylabel("φ")

# TODO: compare ΩΛ with same conventions as Ωr, Ωm
Ωr_cola = data_cola["OmegaR"] + data_cola["OmegaNu"]
Ωr_class = (data_class["9:(.)rho_g"] + data_class["12:(.)rho_ur"]) / data_class["14:(.)rho_tot"] # rho_tot in class is b+c+h+Λ (not smg)
Ωm_cola = data_cola["OmegaM"]
Ωm_class = (data_class["10:(.)rho_b"] + data_class["11:(.)rho_cdm"]) / data_class["14:(.)rho_tot"]
ΩΛ_cola = data_cola["OmegaLambda"]
ΩΛ_class = read_number(f"{dir}/class/log.txt", "Lambda = ") / E_class**2 # TODO: ???
ax3.plot(np.log10(a_cola), Ωr_cola, linestyle = "solid", color = "C0", alpha = alpha, label = "s = r")
ax3.plot(np.log10(a_class), Ωr_class, linestyle = "dashed", color = "C0", alpha = alpha)
ax3.plot(np.log10(a_cola), Ωm_cola, linestyle = "solid", color = "C1", alpha = alpha, label = "s = m")
ax3.plot(np.log10(a_class), Ωm_class, linestyle = "dashed", color = "C1", alpha = alpha)
ax3.plot(np.log10(a_cola), ΩΛ_cola, linestyle = "solid", color = "C2", alpha = alpha, label = "s = Λ")
ax3.plot(np.log10(a_class), ΩΛ_class, linestyle = "dashed", color = "C2", alpha = alpha)
ax3.set_xlabel("log10(a)")
ax3.set_ylabel("ρₛ / ρ")
ax3.legend()
ax3.set_ylim(0, 1);

In [ ]:
def plot_power_spectrum(dir):
    fig, ax = plt.subplots()

    data = np.loadtxt(f"{dir}/class/_pk.dat")
    k, P = data[:, 0], data[:, 1]
    ax.plot(np.log10(k), np.log10(P), marker = ".", label = "linear")

    data = np.loadtxt(f"{dir}/cola/output/pofk_cola_cb_z0.000.txt")
    k, P = data[:, 0], data[:, 1]
    ax.plot(np.log10(k), np.log10(P), marker = ".", label = "nonlinear")
    
    ax.legend()
    return ax

plot_power_spectrum(dir)

## Parameter space (Latin hypercube)

In [ ]:
bounds = {
    # similar to Bacco bounds in https://arxiv.org/pdf/2207.06437 equation (14)
    "ω":  (1e1, 1e4), # TODO: log?
    "Ωm": (0.23, 0.40),
    "σ8": (0.65, 0.90),
    "Ωb": (0.04, 0.06),
    "ns": (0.92, 1.01),
    "h":  (0.60, 0.80),
    "Mν": (0.0, 0.4),
    "w0": (-1.15, -0.85),
    "wa": (-0.3, +0.3),
    #"a": (0.2, 1.0), # TODO: redshift range?
}

columns = bounds.keys() # column names
lo = [bounds[p][0] for p in bounds] # lower bounds
hi = [bounds[p][1] for p in bounds] # upper bounds

sampler = scipy.stats.qmc.LatinHypercube(d=len(bounds), seed=123) # fixed seed for reproducibility
samples = sampler.random(n=10) # draw
samples = scipy.stats.qmc.scale(samples, lo, hi)
samples = pandas.DataFrame(data=samples, columns=columns)
samples

In [ ]:
seaborn.pairplot(samples, corner=True, diag_kind="hist")

In [ ]:
samples.apply(simulate, axis=1); # simulate each parameter sample